# Recursive Linked Data Structures

# Binary Search Trees

## BSTs are like dynamic binary search

In [2]:
def binary_search(A, x):
    """If A is sorted, finds the index of x or the next largest item."""
    # Focus on A[i], ..., A[j] - initially the whole array
    i = 0
    j = len(A)-1
    while i <= j:
        m = (i + j) // 2  # middle index
        if A[m] == x:
            return m
        elif A[m] < x:
            i = m+1
        else:
            j = m-1
    return i  # successor of x; j would give predecessor of x

assert binary_search([1, 3, 4, 5, 9, 11, 23], 5) == 3
assert binary_search([1, 3, 4, 5, 9, 11, 23], 4.5) == 3
assert binary_search([1, 3, 4, 5, 9, 11, 23], 4) == 2
assert binary_search([1, 3, 4, 5, 9, 11, 23], 6) == 4

In [2]:
class BST:
    def __init__(self, key, value, left=None, right=None):
        self.key = key
        self.value = value
        self.left = left
        self.right = right

def example():
    return BST(5, 'D',
        BST(3, 'B',
           BST(1, 'A'),
           BST(4, 'C')
        ),
        BST(11, 'F',
           BST(9, 'E'),
           BST(23, 'G')
        )
    )

print(example())

## Rendering to string

In [6]:
class BST(BST):
    def __str__(self):
        def recurse(node, indent=0):
            prefix = '  ' * indent
            if node is None:
                return prefix + "None"
            left = '\n' + recurse(node.left, indent+1)
            right = '\n' + recurse(node.right, indent+1)
            return f"{prefix}{node.key}: {node.value}" + \
                left + right
            # str(self.key) + ": " + str(self.value)
        return recurse(self)

print(example())

5: D
  3: B
    1: A
      None
      None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None


## Binary Search in a Binary Search Tree

In [9]:
class BST(BST):
    def get(self, key, missing=None):
        """Get the value associated with given key,
        or if key isn't there, return missing."""
        if key == self.key:
            return self.value
        elif key < self.key:
            #BST.get(self.left, key, missing)
            if self.left is None:
                return missing
            else:
                return self.left.get(key, missing)
        else: #key > self.key
            if self.right is None:
                return missing
            else:
                return self.right.get(key, missing)

In [17]:
class BST(BST):
    def get(self, key, missing=None):
        """Get the value associated with given key,
        or if key isn't there, return missing."""
        node = self
        while node is not None:
            if key == node.key:
                return node.value
            elif key < node.key:
                node = node.left
            else: #key > node.key
                node = node.right
        return missing

In [18]:
assert example().get(0) == None
assert example().get(1) == 'A'
assert example().get(2) == None
assert example().get(3) == 'B'
assert example().get(4) == 'C'
assert example().get(4.5) == None
assert example().get(5) == 'D'

In [15]:
class BST(BST):
    def path(self, key):
        """Return a list of node values visited along the way
        to find key"""
        out = []
        def recurse(node):
            out.append(node.value)
            if key == node.key:
                return node.value
            elif key < node.key:
                #BST.get(node.left, key, missing)
                if node.left is not None:
                    return recurse(node.left)
            else: #key > node.key
                if node.right is not None:
                    return recurse(node.right)
        recurse(self)
        return out

In [16]:
example().path(4.5)

['D', 'B', 'C']

In [ ]:
example().path(9)

## Inserting a node

In [20]:
class BST(BST):
    def insert(self, key, value):
        """Add node with specified key and value into BST,
        or overwrite existing node with specified key,
        and return the node."""
        node = self
        while node is not None:
            if key == node.key:
                node.value = value
                return node
            elif key < node.key:
                if node.left is None:
                    node.left = BST(key, value)
                    return node.left
                else:
                    node = node.left
            else: #key > node.key
                if node.right is None:
                    node.right = BST(key, value)
                    return node.right
                else:
                    node = node.right

In [21]:
for key in [0, 1, 2]:
    print(f"** Example after inserting {key}:")
    tree = example()
    tree.insert(key, 'Z')
    print(tree)

** Example after inserting 0:
5: D
  3: B
    1: A
      0: Z
        None
        None
      None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None
** Example after inserting 1:
5: D
  3: B
    1: Z
      None
      None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None
** Example after inserting 2:
5: D
  3: B
    1: A
      None
      2: Z
        None
        None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None


## dict interface

In [32]:
class Missing: pass

class BST(BST):
    def __getitem__(self, key):
        value = self.get(key, Missing)
        if value is not Missing:
            return value
        else:
            raise KeyError(key)
    def __setitem__(self, key, value):
        self.insert(key, value)
    def __contains__(self, key):
        value = self.get(key, Missing)
        return value is not Missing
        #if value is None:
        #    return False
        #else:
        #    return True

In [33]:
tree = example()
print(1 in tree)
print(tree[1])

True
A


In [34]:
tree[1] = 'AAA'
print(tree[1])

AAA


In [35]:
print(2 in tree)
tree[2] = None
print(2 in tree)
print(tree[2])

False
True
None


In [31]:
tree[999]

KeyError: 999

In [26]:
print(tree)

5: D
  3: B
    1: AAA
      None
      2: AB
        None
        None
    4: C
      None
      None
  11: F
    9: E
      None
      None
    23: G
      None
      None


## In-order traversal

In [36]:
class BST(BST):
    def print_in_order(self):
        """Print all (key, value) pairs in order by key"""
        if self.left is not None:
            self.left.print_in_order()
        print(f"{self.key}: {self.value}")
        if self.right is not None:
            self.right.print_in_order()

In [37]:
example().print_in_order()

1: A
3: B
4: C
5: D
9: E
11: F
23: G


In [42]:
class BST(BST):
    def items(self, out = None):
        """Return a list of (key, value) pairs in order by key"""
        if out is None:
            out = []
        #out = []
        #def recuse(node):
        if self.left is not None:
            self.left.items(out)
        out.append((self.key, self.value))
        if self.right is not None:
            self.right.items(out)
        return out

In [47]:
class BST(BST):
    def items(self):
        """Return a list of (key, value) pairs in order by key"""
        out = []
        def recurse(node):
            if node.left is not None:
                recurse(node.left)
            out.append((node.key, node.value))
            if node.right is not None:
                recurse(node.right)
        recurse(self)
        return out

In [49]:
example().items()

[(1, 'A'), (3, 'B'), (4, 'C'), (5, 'D'), (9, 'E'), (11, 'F'), (23, 'G')]

In [50]:
total = 0
for key, value in example().items():
    total += key
print("Total key value:", total)

Total key value: 56


## Iterators and Generators

### Iterators

In [51]:
it = iter([1,2,3])
it

In [52]:
it.__next__()

1

In [53]:
it.__next__()

2

In [54]:
it.__next__()

3

In [55]:
it.__next__()

StopIteration: 

### for loops over iterators

In [56]:
it = iter([1,2,3])
for x in it:
    print(x)
for x in it:
    print(x, 'again')

1
2
3


In [ ]:
list(iter([1,2,3])) == [1,2,3]

In [57]:
class BST(BST):
    def __iter__(self):
        return iter(self.items())

In [58]:
total = 0
for key, value in example():
    total += key
print("Total key value:", total)

Total key value: 56


### Custom iterators via classes

In [ ]:
class IterRange:
    """Single iteration from start to end"""
    def __init__(self, start, end):
        self.value = start
        self.end = end
    def __next__(self):
        value = self.value
        if value < self.end:
            self.value += 1
            return value
        else:
            raise StopIteration()
    def __iter__(self):
        return self  # this object is its own iterator (needed for 'for')

In [ ]:
for x in IterRange(1, 4):
    print(x)

### Custom iterators via generators

In [59]:
def OneTwoThree():
    yield 1
    yield 2
    yield 3
    #implicit 'return None' which stops the iteration

In [60]:
OneTwoThree

<function __main__.OneTwoThree>

In [61]:
OneTwoThree()

<generator object OneTwoThree at 0x00000209FF5B5AF0>

In [62]:
it = OneTwoThree()
for x in it:
    print(x)
for x in it:
    print(x, 'again')

1
2
3


In [ ]:
def IterRange(start, end):
    value = start
    while value < end:
        yield value
        value += 1
    #implicit 'return None' which stops the iteration

In [ ]:
it = IterRange(1, 4)
for x in it:
    print(x)
for x in it:
    print(x, 'again')

### Generator-based BST iteration

In [ ]:
class BST(BST):
    def __iter__(self):
        """Return an iterator of (key, value) pairs in order by key"""
        raise NotImplemented

In [ ]:
total = 0
for key, value in example():
    total += key
print("Total key value:", total)

## Finding nearby elements (inexact search)

In [ ]:
class BST(BST):
    def nearby(self, key):
        """Find a node containing the key, or if it's not present,
        either the next smaller key or the next larger key."""
        raise NotImplemented
    def __repr__(self):
        """Short format for node that doesn't recurse.
        This gets called by tuple.__str__, for example."""
        return f"<BST {self.key}: {self.value}>"

In [ ]:
example().nearby(0)

In [ ]:
example().nearby(1)

In [ ]:
example().nearby(2)

In [ ]:
example().nearby(4.5)

## Deletion

In [ ]:
class BST(BST):
    def delete(self, key, parent=None):
        """Delete node with specified key, if there is one.
        Return whether a node was deleted."""
        raise NotImplemented

    def __delitem__(self, key): # dict iterface
        if not self.delete(key):
            raise KeyError(key)

In [ ]:
tree = example()
del tree[0]
print(tree)

In [ ]:
del tree[5]
print(tree)

In [ ]:
del tree[11]
print(tree)

In [ ]:
del tree[23]
print(tree)

## Limitations

In [ ]:
tree = BST(1, 'A')
del tree[1]